In [23]:
import pandas as pd
import numpy as np
import random, string
from collections import Counter
from unidecode import unidecode

# customer table

In [2]:
random.seed(123)
n = 1329 #ilość klientów

GUS: https://dane.gov.pl/pl/dataset/219,imiona-nadawane-dzieciom-w-polsce/resource/44824/table?page=1&per_page=20&q=&sort=

In [3]:
n_k = pd.read_csv('nazwiska_k.csv', sep = ",")
n_m = pd.read_csv('nazwiska_m.csv', sep = ",")
i_k = pd.read_csv('imiona_k.csv', sep = ",")
i_k = i_k.rename(columns={'LICZBA WYSTĄPIENIEŃ': 'Liczba'})
i_m = pd.read_csv('imiona_m.csv', sep = ",")
i_m = i_m.rename(columns={'LICZBA WYSTĄPIEŃ': 'Liczba'})

In [4]:
N_k = [n_k["Nazwisko aktualne"][i].lower().capitalize() for i in range(150000)]
N_m = [n_m["Nazwisko aktualne"][i].lower().capitalize() for i in range(150000)]
I_k = [i_k["IMIĘ PIERWSZE"][i].lower().capitalize() for i in range(i_k.shape[0])]
I_m = [i_m["IMIĘ PIERWSZE"][i].lower().capitalize() for i in range(i_m.shape[0])]

In [5]:
def normalise(i_x, I_x):
    L_I_x = [i_x["Liczba"][i]/np.sum(i_x["Liczba"]) for i in range(len(I_x))]
    L_I_x = L_I_x/ min(L_I_x)
    L_I_x = L_I_x/ np.sum(L_I_x)
    return L_I_x

In [6]:
first_name = np.zeros(n).astype(str)
last_name = np.zeros(n).astype(str)
gender = np.zeros(n).astype(str)

x = np.random.choice(["K", "M"], size=n, replace=True, p=[0.53, 0.47]) # dla odpowiedniego mianownika kobiet do mężczyzn
imiona_m = np.random.choice(I_m, size=n, replace=True, p=normalise(i_m, I_m))
imiona_k = np.random.choice(I_k, size=n, replace=True, p=normalise(i_k, I_k))
nazwiska_m = np.random.choice(N_m, size=n, replace=True, p=normalise(n_m, N_m))
nazwiska_k = np.random.choice(N_k, size=n, replace=True, p=normalise(n_k, N_k))

for i in range(n):
    if x[i] == "K":
        first_name[i] = imiona_k[i] #random.choice(I_k)
        last_name[i] = nazwiska_k[i] #random.choice(N_k)
        gender[i] = "female"
    else:
        first_name[i] = imiona_m[i] #random.choice(I_m)
        last_name[i] = nazwiska_m[i] #random.choice(N_m)
        gender[i] = "male"

In [7]:
customer_id = range(1,n+1)
dict_customer = {'customer_id':customer_id, "first_name": first_name, "last_name": last_name, "gender": gender}
customer = pd.DataFrame(dict_customer)

In [8]:
customer #1329 osób

,customer_id,first_name,last_name,gender
0,1,Ewa,Sapak,female
1,2,Jakub,Szczepański,male
2,3,Kateryna,Teliszewska,female
3,4,Nela,Dendys,female
4,5,Maja,Zdanowska,female
...,...,...,...,...
1324,1325,Nela,Zgierska,female
1325,1326,Maciej,Kuryshchuk,male
1326,1327,Kazimierz,Owczarek,male
1327,1328,Julian,Rozmus,male


https://interaktywnie.com/biznes/artykuly/biznes/przeglad-ktora-poczta-e-mail-jest-najlepsza-16950

In [9]:
# liczby użytkowników danych poczt email
gmail = 8968134
wp = 6895602
onet = 4302509
interia = 2677345
o2 = 2855010
pwr = 21902

all_users = gmail+wp+onet+interia+o2+pwr

# wektor prawdopodobieństw
mail_probab = [gmail/all_users, wp/all_users, onet/all_users, interia/all_users, o2/all_users, pwr/all_users]
mail_probab = [round(i,4) for i in mail_probab]
mail_probab[3] = 0.1040

In [10]:
email = np.zeros(n).astype(str)


for i in range(n):
    
    x = random.choice(range(1,3))
    
    if x == 1:
        nazwa = customer["first_name"][i]
        y = random.choice(range(1,5))
        if y == 1:
            nazwa += "." + customer["last_name"][i]
        elif y == 2:
            nazwa += customer["last_name"][i]
        elif y == 3:
            nazwa += "." + customer["last_name"][i][0:random.choice(range(3,5))]
        else:
            nazwa += str(random.choice(range(100,10000)))
        
    else:
        nazwa = customer["last_name"][i] 
        y = random.choice(range(1,5))
        if y == 1:
            nazwa += "." + customer["first_name"][i]
        elif y == 2:
            nazwa += customer["first_name"][i]
        elif y == 3:
            nazwa += "." + customer["first_name"][i][0:random.choice(range(2,5))]
        else:
            nazwa += str(random.choice(range(100,10000)))
        
    email[i] = nazwa
    
    
domeny = ["@gmail.com", "@wp.pl", "@onet.pl", "@interia.pl", "@o2.pl", "@student.pwr.edu.pl"]
domena = np.random.choice(domeny, size=len(email), replace=True, p=mail_probab)    

for i in range(n):
    email[i] += domena[i]
email = [unidecode(i) for i in email] 

https://pl.wikipedia.org/wiki/Numery_telefoniczne_w_Polsce

In [11]:
phone = np.zeros(n).astype(int)
for i in range(n):
    number =  str(np.random.choice([45, 50, 51, 53, 57, 60, 66, 69, 72, 73, 78, 79, 88])) + str(random.choice(range(1000000, 9000000)))
    phone[i] = int(number)
customer["phone"] = phone
customer["email"] = email
customer

,customer_id,first_name,last_name,gender,phone,email
0,1,Ewa,Sapak,female,791491634,Ewa.Sap@gmail.com
1,2,Jakub,Szczepański,male,792370041,Szczepanski.Ja@gmail.com
2,3,Kateryna,Teliszewska,female,663017148,Kateryna8885@wp.pl
3,4,Nela,Dendys,female,734919744,Dendys.Ne@gmail.com
4,5,Maja,Zdanowska,female,577698695,MajaZdanowska@onet.pl
...,...,...,...,...,...,...
1324,1325,Nela,Zgierska,female,503904401,Zgierska7432@wp.pl
1325,1326,Maciej,Kuryshchuk,male,538455331,Maciej.Kur@gmail.com
1326,1327,Kazimierz,Owczarek,male,781806955,Kazimierz.Owczarek@wp.pl
1327,1328,Julian,Rozmus,male,535851916,Julian4145@gmail.com


In [12]:
counter = Counter(phone)
result = [i for i, j in counter.items() if j > 1]
if result != []:
    print("Te same numery telefonu!!")
else:
    print("phone: Wszystko ok.")
    
counter = Counter(last_name)
result = [i for i, j in counter.items() if j > 1]
if result != []:
    print("Te same nazwiska!!")
else:
    print("last_name: Wszystko ok.")

phone: Wszystko ok.
Te same nazwiska!!


In [13]:
print(counter)

Counter({'Nowak': 8, 'Kwiatkowska': 6, 'Kowalczyk': 6, 'Kowalski': 5, 'Szewczyk': 5, 'Woźniak': 4, 'Walczak': 4, 'Wiśniewska': 4, 'Kowalska': 4, 'Wieczorek': 3, 'Lis': 3, 'Mazur': 3, 'Trawińska': 3, 'Malinowska': 3, 'Janus': 3, 'Krawczyk': 3, 'Maćkowiak': 3, 'Pawłowska': 3, 'Dąbrowska': 3, 'Gil': 3, 'Wysocki': 3, 'Kozłowska': 3, 'Jędrzejewski': 3, 'Szczepański': 2, 'Wojnar': 2, 'Góra': 2, 'Bąk': 2, 'Polak': 2, 'Kwiatkowski': 2, 'Głuch': 2, 'Małecki': 2, 'Przybyła': 2, 'Kozicka': 2, 'Król': 2, 'Stec': 2, 'Michalski': 2, 'Majda': 2, 'Weber': 2, 'Krzewska': 2, 'Pawlikowski': 2, 'Gromek': 2, 'Kozieł': 2, 'Jankowska': 2, 'Bobrek': 2, 'Barańska': 2, 'Stankiewicz': 2, 'Chojnacka': 2, 'Bujanowski': 2, 'Zawadzka': 2, 'Kasperek': 2, 'Dudek': 2, 'Pietrzak': 2, 'Kołodziej': 2, 'Przybylski': 2, 'Kopacz': 2, 'Zych': 2, 'Janicka': 2, 'Czarnecka': 2, 'Domańska': 2, 'Kamińska': 2, 'Kramarz': 2, 'Stanisławski': 2, 'Kasprzak': 2, 'Wnuk': 2, 'Wróbel': 2, 'Dukat': 2, 'Zielińska': 2, 'Gruszczyńska': 2, 'Ada

In [14]:
#customer = customer.drop('', axis=1)
customer.reset_index(drop=False, inplace=False)

customer

,customer_id,first_name,last_name,gender,phone,email
0,1,Ewa,Sapak,female,791491634,Ewa.Sap@gmail.com
1,2,Jakub,Szczepański,male,792370041,Szczepanski.Ja@gmail.com
2,3,Kateryna,Teliszewska,female,663017148,Kateryna8885@wp.pl
3,4,Nela,Dendys,female,734919744,Dendys.Ne@gmail.com
4,5,Maja,Zdanowska,female,577698695,MajaZdanowska@onet.pl
...,...,...,...,...,...,...
1324,1325,Nela,Zgierska,female,503904401,Zgierska7432@wp.pl
1325,1326,Maciej,Kuryshchuk,male,538455331,Maciej.Kur@gmail.com
1326,1327,Kazimierz,Owczarek,male,781806955,Kazimierz.Owczarek@wp.pl
1327,1328,Julian,Rozmus,male,535851916,Julian4145@gmail.com


In [15]:
customer.to_csv("customer.csv", sep = ",", encoding='utf-8-sig')

# staff table

In [31]:
random.seed(123)
n = 17 #ilość pracowników

In [32]:
first_name = np.zeros(n).astype(str)
last_name = np.zeros(n).astype(str)
gender = np.zeros(n).astype(str)

x = np.random.choice(["K", "M"], size=n, replace=True, p=[0.35, 0.65]) # chcemy więcej mężczyzn jako pracowników
imiona_m = np.random.choice(I_m, size=n, replace=True, p=normalise(i_m, I_m))
imiona_k = np.random.choice(I_k, size=n, replace=True, p=normalise(i_k, I_k))
nazwiska_m = np.random.choice(N_m, size=n, replace=True, p=normalise(n_m, N_m))
nazwiska_k = np.random.choice(N_k, size=n, replace=True, p=normalise(n_k, N_k))

for i in range(n):
    if x[i] == "K":
        first_name[i] = imiona_k[i] #random.choice(I_k)
        last_name[i] = nazwiska_k[i] #random.choice(N_k)
        gender[i] = "female"
    else:
        first_name[i] = imiona_m[i] #random.choice(I_m)
        last_name[i] = nazwiska_m[i] #random.choice(N_m)
        gender[i] = "male"

In [33]:
staff_id = range(1,n+1)
dict_staff = {'staff_id': staff_id, "first_name": first_name, "last_name": last_name, "gender": gender}
staff = pd.DataFrame(dict_staff)
staff

,staff_id,first_name,last_name,gender
0,1,Witold,Krzepkowski,male
1,2,Oliwier,Łabuński,male
2,3,Błażej,Janowski,male
3,4,Wiktor,Małecki,male
4,5,Beniamin,Renkowski,male
5,6,Marcel,Kostrubiec,male
6,7,Wojciech,Antosiewicz,male
7,8,Maksymilian,Ignaczak,male
8,9,Maksymilian,Mendrala,male
9,10,Mikołaj,Lebiodzik,male


In [34]:
# liczby użytkowników danych poczt email
gmail = 8968134
wp = 6895602
onet = 4302509
interia = 2677345
o2 = 2855010

all_users = gmail+wp+onet+interia+o2

# wektor prawdopodobieństw
mail_probab = [gmail/all_users, wp/all_users, onet/all_users, interia/all_users, o2/all_users]
mail_probab = [round(i,4) for i in mail_probab]
mail_probab = [i/min(mail_probab) for i in mail_probab]
mail_probab = mail_probab/np.sum(mail_probab)

In [35]:
email = np.zeros(n).astype(str)
for i in range(n):
    email[i] = staff["first_name"][i] + "." + staff["last_name"][i] + "@dragons.com"
email = [unidecode(i) for i in email] 

In [36]:
phone = np.zeros(n).astype(int)
for i in range(n):
    number =  str(np.random.choice([45, 50, 51, 53, 57, 60, 66, 69, 72, 73, 78, 79, 88])) + str(random.choice(range(1000000, 9000000)))
    phone[i] = int(number)
staff["phone"] = phone
staff["email"] = email
staff

amount = [4310, 5140, 6530, 7280, 8320] #ceny brutto
salary = np.random.choice(amount, size=n, replace=True, p=[0.35, 0.35, 0.15, 0.10, 0.05]). astype(float)
staff["salary"] = salary 
staff

,staff_id,first_name,last_name,gender,phone,email,salary
0,1,Witold,Krzepkowski,male,531439257,Witold.Krzepkowski@dragons.com,6530.0
1,2,Oliwier,Łabuński,male,883245438,Oliwier.Labunski@dragons.com,5140.0
2,3,Błażej,Janowski,male,511731374,Blazej.Janowski@dragons.com,5140.0
3,4,Wiktor,Małecki,male,607450473,Wiktor.Malecki@dragons.com,5140.0
4,5,Beniamin,Renkowski,male,794416191,Beniamin.Renkowski@dragons.com,5140.0
5,6,Marcel,Kostrubiec,male,883236015,Marcel.Kostrubiec@dragons.com,6530.0
6,7,Wojciech,Antosiewicz,male,501903455,Wojciech.Antosiewicz@dragons.com,5140.0
7,8,Maksymilian,Ignaczak,male,668033877,Maksymilian.Ignaczak@dragons.com,4310.0
8,9,Maksymilian,Mendrala,male,608559804,Maksymilian.Mendrala@dragons.com,5140.0
9,10,Mikołaj,Lebiodzik,male,458338570,Mikolaj.Lebiodzik@dragons.com,4310.0


In [37]:
staff.to_csv("staff.csv", sep = ",", encoding='utf-8-sig')